In [ ]:
import os

import dotenv
from agents import (
    Agent,
    GuardrailFunctionOutput,
    InputGuardrail,
    InputGuardrailTripwireTriggered,
    Runner,
    RunResult,
    set_default_openai_api,
    set_default_openai_client,
)
from openai import AsyncAzureOpenAI, AsyncOpenAI
from pydantic import BaseModel


OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export


In [4]:
dotenv.load_dotenv()
use_local_model = True

azure_client = AsyncAzureOpenAI(
    base_url=os.environ.get("AZURE_AOI_BASE_URL"),
    api_version=os.environ.get("AZURE_OAI_API_VERSION"),
    api_key=os.environ.get("AZURE_OAI_TOKEN"),
    azure_deployment=os.environ.get("AZURE_OAI_DEPLOYMENT_NAME")
)

ollama_client = AsyncOpenAI(
    base_url="http://host.docker.internal:11434/v1",
    api_key="ollama",
)

set_default_openai_api("chat_completions")

if use_local_model:
    set_default_openai_client(ollama_client, use_for_tracing=False)
    model = "llama3.1:latest"
else:
    set_default_openai_client(azure_client, use_for_tracing=False)
    model = "gpt-4o"

In [5]:
use_guardrails = True


class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str


async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    model=model,
    output_type=HomeworkOutput,
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    model=model,
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    model=model,
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)


triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent],
    model=model,
    input_guardrails=[InputGuardrail(
        guardrail_function=homework_guardrail)] if use_guardrails else [],
)

In [ ]:
async def main():
    try:
        result: RunResult = await Runner.run(
            starting_agent=triage_agent,
            input="who was the first president of the united states?",
            max_turns=5)
        print(result.final_output)
    except InputGuardrailTripwireTriggered as e:
        print(f"Guardrail triggered: {e.guardrail_result.output.output_info}")

In [7]:
await main()

Guardrail triggered: is_homework=False reasoning='The question appears to be a general knowledge or trivia question, rather than a request for help with completing homework.'
